# Extend MNIST with plus and minus signs

To create "+" and "-" sign images that match the aesthetics of the MNIST digit images we used the first 8208 images of the "+" folder of the [Handwritten math symbols](https://www.kaggle.com/datasets/xainano/handwrittenmathsymbols) dataset from Kaggle as well as the first 11251 images of the "-" folder. Since the neither stroke size nor width and height of the images matched the MNIST dataset some simple preprossing using OpenCV was necessary. The images as well as the MNIST digit images were saved to .jpg format and put into a PyTorch data folder structure.

In [1]:
# Import OpenCV for image processing
import cv2
from google.colab.patches import cv2_imshow # only necessary in Colab to show the OpenCV images
# Import some os methods to get information about the image locations
from os import listdir
from os.path import join
import numpy as np
import math

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Functions to process the signs

In [3]:
# Returns the index of the first row containing a non-zero value (pixel)
def get_upper_border(img):
  for i in range(len(img)):
    for j in range(len(img[i])):
      if not img[i][j] == 0:
          return i

# Returns the index of the last row containing a non-zero value (pixel)
def get_lower_border(img):
  for i in range(len(img)-1, 0, -1):
    for j in range(len(img[i])):
      if not img[i][j] == 0:
          return i

# Returns the index of the first column containing a non-zero value (pixel)
def get_left_border(img):
  for j in range(len(img[0])):
    for i in range(len(img)):
      if not img[i][j] == 0:
          return j

# Returns the index of the last column containing a non-zero value (pixel)
def get_right_border(img):
  for j in range(len(img[0])-1, 0, -1):
    for i in range(len(img)):
      if not img[i][j] == 0:
          return j

In [4]:
# Returns the image without any white space around the digit
def get_img(img_sample):
  upper_border = get_upper_border(img_sample)
  lower_border = get_lower_border(img_sample)
  left_border = get_left_border(img_sample)
  right_border = get_right_border(img_sample)

  img_reduced_rows = img_sample[upper_border:lower_border+1]
  img = []
  for row in img_reduced_rows:
    img.append(row[left_border:right_border+1])
  
  return img

In [5]:
# Preprocesses the "+" and "-" sign images to look more like the MNIST digit images
def preprocess_signs(img):
  img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # grayscale the image
  img = cv2.bitwise_not(img)
  # Apply morphological dilation twice to thicken the stroke of the signs
  for _ in range(2):
    for i in range(1, img.shape[0]-1):
      for j in range(1, img.shape[1]-1):
        values = np.array([img[i,j], img[i,j+1], img[i+1,j-1], img[i+1,j], img[i+1,j+1]])
        img[i,j] = np.max(values)
  # Scale the images down, so that it fits in the 28x28 px of MNIST images
  img = cv2.pyrDown(img)
  return img

In [6]:
# Adds whitespace to every side of the sign, so that it fits the MNIST width and height
def fill_whitespace(img, width=28, height=28):
  fill_x = math.floor((width - img.shape[1]) / 2)
  fill_y = math.ceil((height - img.shape[0]) / 2)
  new_img = np.zeros((width, height))
  new_img[fill_y: fill_y + img.shape[0], fill_x: fill_x + img.shape[1]] = img
  return new_img

## Load the original data

In [7]:
input_path = '/content/drive/MyDrive/dlproject'

In [8]:
plus_sign_path = join(input_path, 'plus-signs')
minus_sign_path = join(input_path, 'minus-signs')

In [9]:
plus_indices = listdir(plus_sign_path)
minus_indices = listdir(minus_sign_path)

In [10]:
len(plus_indices)

8208

In [11]:
len(minus_indices)

11251

## Create the dataset

In [ ]:
# Process the first 6000 images of the "+" sign part of the Kaggle Math Symbols dataset to create a training dataset
for plus in enumerate(plus_indices[:6000]):
  index = plus[0]
  plus_index = plus[1]
  img = cv2.imread(join(plus_sign_path, plus_index))
  img = preprocess_signs(img)
  img = fill_whitespace(img)
  cv2.imwrite(join('/content/drive/MyDrive/deeplearning/sign-images/train-plus-signs', '+_' + str(index)) + '.jpg', img)

In [ ]:
# Process the first 6000 images of the "-" sign part of the Kaggle Math Symbols dataset to create a training dataset
for minus in enumerate(minus_indices[:6000]):
  index = minus[0]
  minus_index = minus[1]
  img = cv2.imread(join(minus_sign_path, minus_index))
  img = preprocess_signs(img)
  img = fill_whitespace(img)
  cv2.imwrite(join('/content/drive/MyDrive/deeplearning/sign-images/train-minus-signs', '-_' + str(index)) + '.jpg', img)

In [ ]:
# Create a test/validation dataset for the "+" signs
for plus in enumerate(plus_indices[6000:7000]):
  index = plus[0]
  plus_index = plus[1]
  img = cv2.imread(join(plus_sign_path, plus_index))
  img = preprocess_signs(img)
  img = fill_whitespace(img)
  cv2.imwrite(join('/content/drive/MyDrive/deeplearning/sign-images/test-plus-signs', '+_' + str(index)) + '_test.jpg', img)

In [ ]:
# Create a test/validation dataset for the "-" signs
for minus in enumerate(minus_indices[6000:7000]):
  index = minus[0]
  minus_index = minus[1]
  img = cv2.imread(join(minus_sign_path, minus_index))
  img = preprocess_signs(img)
  img = fill_whitespace(img)
  cv2.imwrite(join('/content/drive/MyDrive/deeplearning/sign-images/test-minus-signs', '-_' + str(index)) + '_test.jpg', img)